# Drifting y Quality

Un desafío común para los **data scientists** en el mundo real es que, a diferencia de las competencia de **Kaggle**, los datos cambian con el tiempo. Esto significa que los datos que recibiremos en el futuro no necesariamente reflejarán los que usamos para entrenar nuestros modelos. Esta homogeneidad es esencial, ya que los algoritmos aprenden patrones en función de las distribuciones observadas durante el entrenamiento. Si esas distribuciones cambian, la capacidad predictiva del modelo se deteriora.

Podemos identificar dos tipos principales de problemáticas asociadas a estos cambios:

- **Data Quality (DQ)**
- **Data Drift (DF)**

Comencemos con el primero: **Data Quality (DQ)**. Hay mucho que decir sobre este tema, pero mencionaremos brevemente que los datos provienen de procesos creados por seres humanos, quienes son propensos a cometer errores y hacer modificaciones. A lo largo del ciclo de vida de los modelos, es común que se enfrenten a este tipo de situaciones. Por ello, es fundamental implementar controles que garanticen que los datos a los que se aplican los modelos mantienen la misma estructura que los datos usados en el entrenamiento.

A continuación, veremos dos indicadores clave para monitorear la calidad de los datos, aunque no son los únicos a considerar.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from os import path

In [2]:
base_path = 'C:/Eugenio/Maestria/DMEyF/'
dataset_path = base_path + 'datasets/'
dataset_file = 'competencia_01.csv'

data = pd.read_csv(path.join(dataset_path,dataset_file))

Trabajaremos con un mes de entrenamiento, pero pueden ser muchos más y el mes donde finalmente haremos la predicción.

In [3]:
mes_train = 202104
mes_score = 202106


El primer control es la cantidad de **valores nulos** por variable. Cuando ocurre un error en el proceso de generación de datos, es común que se manifieste un aumento en la cantidad de valores nulos.


In [4]:
train_data = data[data['foto_mes'] == mes_train]
score_data = data[data['foto_mes'] == mes_score] # este va a ser siempre un solo mes pero de entrenamiento puedo usar varios meses.

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted

,Train Null Percentage,Score Null Percentage,diff
clase_ternaria,0.000000,100.000000,100.000000
Master_mconsumototal,59.780608,58.459691,1.320917
Master_cconsumos,59.780608,58.459691,1.320917
Master_mpagospesos,59.780608,58.459691,1.320917
Master_cadelantosefectivo,59.780608,58.459691,1.320917
...,...,...,...
ccajas_otras,0.000000,0.000000,0.000000
catm_trx,0.000000,0.000000,0.000000
matm,0.000000,0.000000,0.000000
catm_trx_other,0.000000,0.000000,0.000000


Un problema similar es la sobrerrepresentación de ceros en los datos, ya que este valor se utiliza comúnmente para la imputación. La acumulación excesiva de ceros puede ser indicativa de un error o de un proceso de imputación inadecuado.

In [5]:
train_zero_percentage = (train_data == 0).mean() * 100
score_zero_percentage = (score_data == 0).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Zero Percentage': train_zero_percentage, 'Score Zero Percentage': score_zero_percentage})

comparison_df_zero['diff_zero_percentage'] = (comparison_df_zero['Score Zero Percentage'] - comparison_df_zero['Train Zero Percentage']).abs()
diff_zero_percentage_sorted = comparison_df_zero.sort_values('diff_zero_percentage',ascending=False)
diff_zero_percentage_sorted


,Train Zero Percentage,Score Zero Percentage,diff_zero_percentage
Master_fultimo_cierre,0.000000,68.891773,68.891773
Visa_fultimo_cierre,0.000000,68.829909,68.829909
cmobile_app_trx,28.197331,23.683859,4.513471
mcuenta_corriente,47.169236,49.692496,2.523260
mtransferencias_recibidas,26.395880,24.303719,2.092161
...,...,...,...
Visa_Finiciomora,0.000000,0.000000,0.000000
Visa_Fvencimiento,0.000000,0.000000,0.000000
Visa_mlimitecompra,0.000000,0.000000,0.000000
Visa_fechaalta,0.000000,0.000000,0.000000


Nota: Que hacer en el caso de que una variable pasa a tener muchos 0s como el caso de visa_fultimo_cierre. Una opcion es eliminar la varibable, incluso si es una variable importante los modelos son bastante recilientes y no deberia cambiar mucho. Otra opcion es imputar esos 0s con un valor razonable.

La segunda problemática es el **data drifting**, que es un fenómeno que ocurre cuando la distribución de los datos cambia con el tiempo.

Hay varios tipos de drifting:

1. **Feature Drift**: Este tipo de deriva se da cuando cambia la distribución de una feature del modelo.

2. **Concept Drift**: Ocurre cuando cambia la relación entre las features y la variable target. Por ejemplo, en un modelo de predicción de fraude, la forma en que los fraudes ocurren puede cambiar con el tiempo, lo que significa que el modelo necesitaría ser ajustado para reconocer nuevos patrones.

Para detectar el drifting utilizaremos el **PSI (Population Stability Index)** que es una métrica utilizada para medir los cambios en la distribución de una variable a lo largo del tiempo. Funciona cuantificando las diferencias entre dos distribuciones, generalmente comparando los datos de entrenamiento con los datos actuales.

* **Cómo funciona el PSI**:
 1. **División en intervalos (bins)**: Tanto los datos históricos como los datos actuales se dividen en una serie de intervalos o bins.
   
 2. **Cálculo de frecuencias**: Se calcula la proporción de observaciones que caen en cada bin tanto para la distribución original (entrenamiento) como para la nueva (actual).

 3. **Fórmula del PSI**: Para cada bin, se calcula la diferencia entre las frecuencias observadas en ambas distribuciones mediante la siguiente fórmula:

   $
   PSI = \sum \left( (P_{actual} - P_{esperado}) \times \log \left( \frac{P_{actual}}{P_{esperado}} \right) \right)
   $
   Donde:
   - $P_{actual}$ es la proporción de la nueva muestra en un bin.
   - $P_{esperado}$ es la proporción de la muestra original en el mismo bin.



Lo que se puede hacer si hay data drifting es ir reentrenando en una nueva ventana de tiempo, antes se hacian y se dejaban correr por 3 o 4 años lo que era un monton, hoy en dia se van reentrenando seguido para que no pierda la performance.

In [6]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null


Y aplicamos el análisis a (casi) todos los **features**

In [7]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)
psi_df


C:\Users\eaneg\AppData\Local\Temp\ipykernel_3336\2860382255.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\eaneg\AppData\Local\Temp\ipykernel_3336\2860382255.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


,feature,psi
135,Visa_Finiciomora,inf
113,Master_Finiciomora,inf
122,Master_fultimo_cierre,1.129066
144,Visa_fultimo_cierre,0.937571
50,cpayroll_trx,0.241274
...,...,...
97,ccajas_transacciones,0.000000
100,ccajas_extracciones,0.000000
99,ccajas_depositos,0.000000
98,ccajas_consultas,0.000000


Encontramos un par de variables conflictivas **Master_Finiciomora** y **Visa_Finiciomora** que nos dan que hay infinito cambio. Vamos a ver que sucedio, haciendo un poco de **deep dive**.

In [8]:
variable_name = 'Master_Finiciomora'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=10, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')


Cortes en Master_Finiciomora: [np.float64(1.999), np.float64(11.0), np.float64(18.0), np.float64(36.2), np.float64(46.0), np.float64(207.0)]
Frecuencia Esperada: [ 44 103 573  27 282]
Frecuencia Actual: [ 69   0 338   7 245]


* Qué paso? y cómo podría arreglarse?

El resto de las variables que vemos son casos ya detectados. Observamos también que hay un cambio en las variables de **payroll**


In [9]:
variable_name = 'cpayroll_trx'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=20, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')

Cortes en cpayroll_trx: [np.float64(-0.001), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(251.0)]
Frecuencia Esperada: [75458 54661 22952 11019]
Frecuencia Actual: [73677 26686 39300 25212]


Pero antes de tomar una decisión, vamos a analizar como los **lgbm** cortan esa variable, para esto importamos nuestro modelo y analizamos sus puntos de corte por cada variable

In [12]:
model = lgb.Booster(model_file='C:/Eugenio/Maestria/DMEyF/modelos/lgb_first.txt')


In [16]:
model_df = model.trees_to_dataframe()

In [17]:
model_df[model_df['split_feature'] == 'cpayroll_trx'].sort_values('threshold')

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
946,7,2,7-S1,7-S4,7-S9,7-S0,cpayroll_trx,229.067993,1.000000e-35,<=,left,None,0.058128,610.172,25138
981,7,2,7-S2,7-S3,7-S39,7-S0,cpayroll_trx,196.725006,1.000000e-35,<=,left,None,-0.032981,1206.340,137508
1265,9,2,9-S2,9-S4,9-S25,9-S0,cpayroll_trx,134.231995,1.000000e-35,<=,left,None,-0.035770,1030.990,130095
2160,16,1,16-S0,16-S1,16-S10,None,cpayroll_trx,314.158997,1.000000e-35,<=,left,None,0.000000,0.000,162646
2295,17,1,17-S0,17-S1,17-S4,None,cpayroll_trx,285.696991,1.000000e-35,<=,left,None,0.000000,0.000,162646
3915,29,1,29-S0,29-S1,29-S3,None,cpayroll_trx,101.253998,1.000000e-35,<=,left,None,0.000000,0.000,162646
4186,31,2,31-S1,31-S2,31-S5,31-S0,cpayroll_trx,83.286598,1.000000e-35,<=,left,None,-0.002646,1640.440,162527
5535,41,1,41-S0,41-S2,41-S1,None,cpayroll_trx,34.423100,1.000000e-35,<=,left,None,0.000000,0.000,162646
5670,42,1,42-S0,42-S2,42-S1,None,cpayroll_trx,31.052401,1.000000e-35,<=,left,None,0.000000,0.000,162646
5940,44,1,44-S0,44-S2,44-S1,None,cpayroll_trx,25.402201,1.000000e-35,<=,left,None,0.000000,0.000,162646


* En que valores corta la variable **cpayroll_trx**?

Estos análisis fueron hechos entre **Abril** y **Junio**.
+ Qué sucederá con si **Enero** está en el mismo dataset?
+ Cómo puede corregir el drifting si este está presente en una **feature** importante?